# Proyecto Open Data I
## Radares, y su eficiencia en la CAM
### Recopilación, limpieza y tratamiento de los datos
Este cuaderno pretende enseñar el proceso de limpieza de los datos relativos a los radares en la CAM
_Paula Gómez Lucas, Alejandro Majado Martínez_

In [ ]:
# Importar librerías
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import numpy as np
from pandas_summary import DataFrameSummary
import textwrap

from scipy.stats import chisquare

A continuación, se muestra la clase que está compuesta de todos los métodos que se encargan de la limpieza y transformación de los datos

In [ ]:
class CSVDataLoader:
    """
    A class for loading and cleaning CSV data from a specified folder path.

    Attributes:
    -----------
    folder_path : str
        The path to the folder containing the CSV files to be loaded.

    data : dict
        A dictionary containing the loaded CSV data, where the keys are the file names and the values are the corresponding dataframes.
    """

    def __init__(self, folder_path):
        """
        Initializes a CSVDataLoader object with the specified folder path.

        Parameters:
        -----------
        folder_path : str
            The path to the folder containing the CSV files to be loaded.
        """
        self.folder_path = folder_path
        self.data = {}
        self.filename = []
        self.keys = []

    def load_data(self):
        """
        Loads CSV data from the specified folder path into a dictionary.

        Returns:
        --------
        None
        """
        csv_files = [f for f in os.listdir(self.folder_path) if f.endswith('.csv')]
        folders = ("datasets/actuacionesBomberos", "datasets/estaciones", "datasets/accidentalidad")
        for folder in folders:
            df = None
            for file in os.listdir(folder):
                filepath = folder + "/" + file
                df1 = pd.read_csv(filepath, sep=';', encoding='utf-8', low_memory=False)
                df = pd.concat([df, df1])
            self.data[str(folder)] = df

        for file_name in csv_files:
            file_path = os.path.join(self.folder_path, file_name)
            try:
                df = pd.read_csv(file_path, sep=';', encoding='latin-1', low_memory=False)
                self.data[str(file_name)] = df
                self.filename.append(file_name)
            except Exception as e:
                print(f"Error al leer {file_name}: {str(e)}")

        for value in self.data.keys():
            self.keys.append(value)

    def clean_data(self):
        """
        Cleans the loaded CSV data by renaming columns, removing whitespace, dropping null values and duplicates, and converting date columns to datetime format.

        Returns:
        --------
        None
        """
        columna_borrar = "Unnamed"
        for df in self.data:
            for j in self.data[df].columns:
                if columna_borrar in j:
                    while j in self.data[df].columns:
                        self.data[df] = self.data[df].drop(j, axis=1)
                        self.data[df] = self.data[df].dropna(how='all', axis=0)
                        
            self.data[df] = self.data[df].rename(columns = lambda x: x.strip().lower().replace(' ', '_'))
            self.data[df] = self.data[df].map(lambda x: x.strip() if isinstance(x, str) else x)
            self.data[df] = self.data[df].dropna(how='all', axis=0)
            self.data[df] = self.data[df].drop_duplicates()
            self.data[df] = self.data[df].loc[:, ~self.data[df].columns.duplicated()]
            self.data[df].columns = map(str.upper, self.data[df].columns)

            if 'FECHA' in self.data[df].columns:
                self.data[df]['FECHA'] = pd.to_datetime(self.data[df]['FECHA'], format='%d/%m/%Y')

#           num_cols = self.data[i].select_dtypes(include='number').columns
#           for col in num_cols:
#               self.data[i][col] = self.data[i][col].fillna(self.data[i][col].mean())

    def get_info(self, filename):
        print(self.data[filename].isnull().sum())
        print(self.data[filename].info())
        
    def get_nan_columns(self):
        j = 0
        for i in self.data:
            print(self.keys[j])
            self.get_info(i)
            j+=1
      
    def get_cleaned_data(self):
        """
        Returns the cleaned CSV data as a dictionary.

        Returns:
        --------
        dict
            A dictionary containing the cleaned CSV data, where the keys are the file names and the values are the corresponding dataframes.
        """
        return self.data

    def create_graph(df, colummn, name):
        frec = df[''+str(colummn)].value_counts()
        aux_df = pd.DataFrame(frec)
        aux_df.columns = ["Frecuencia absoluta"]
        aux_df["Frecuencia relativa"] = 100*aux_df["Frecuencia absoluta"] / len(df)
        frec_rel_cumsum = aux_df["Frecuencia relativa"].cumsum()
        aux_df["Frecuencia relativa acumulada"] = frec_rel_cumsum
        fig = plt.figure()
        ax = fig.add_subplot(1,1,1)
        ax.set_title('Distribución de '+ str(name))
        ax.bar(aux_df.index, aux_df['Frecuencia absoluta'], color='blue')
        ax2 = ax.twinx()
        ax2.plot(aux_df.index, aux_df['Frecuencia relativa acumulada'], color='red', marker='o', ms = 5)
        ax2.yaxis.set_major_formatter(PercentFormatter())
        ax.tick_params(axis='y', color = 'blue')
        ax2.tick_params(axis='y', color = 'red')
        ax.set_xticklabels(aux_df.index, rotation=90)
        plt.show()

    def dataframe_summary(self, filename):

        numeric_mask = self.data[filename].select_dtypes(include='number').columns

        # Create a DataFrameSummary object
        summary = DataFrameSummary(self.data[filename][numeric_mask])
    
        # Display summary statistics
        summary_stats = summary.summary()
    
        # Plot correlation matrix for all numeric columns
        self.data[filename][numeric_mask].corr(method='pearson', numeric_only=True)

        # Boxplot of each variable
        #self.data[filename][numeric_mask].boxplot(figsize=(10, 8))

        return summary_stats

Una vez está definida la clase con sus métodos, procedemos a declarar las variables que nos permiten trabajar con ello

In [ ]:
folder_path = "datasets"
data_loader = CSVDataLoader(folder_path)

La siguiente función carga los datos de los csv a los dataframes

In [ ]:
data_loader.load_data()

Limpiamos los datos eliminando las columnas autogeneradas con NaNs, renombramos las columnas para que sean uniformes (minúsculas y con barra bajas), eliminamos las filas de NaNs, eliminamos las filas duplicadas, formateamos todas las variables fecha para que sean consistentes (dd/mm/aaaa), sustituimos los NaNs de las variables numéricas con la media correspondiente a su variable.

In [ ]:
data_loader.clean_data()
data = data_loader.get_cleaned_data()

Por último, para confirmar que los datos se han cargado bien, utilizamos el método get_nan_columns para ver cuántos datos en cada columna quedan nulos, así como usamos el método info() de pandas para ver un resumen de todas las columnas y comprobamos también que todo el formateo de las columnas se ha realizado sin problema.

Empecemos con accidentalidad: 

In [ ]:
data_loader.get_info('datasets/accidentalidad')
data_loader.data['datasets/accidentalidad']

Hay algunos datos faltantes que tiene sentido que lo sean, y podemos sustituir por un buzzword de algún tipo que nos haga saber que se trate de esto, como lo es que en un accidente en el que no se han producido lesiones, la lesividad sea nula y tampoco haya código de la misma, y como ambas cifras coinciden, es lógico pensar que se trata de las mismas situaciones. Podemos sustituir entonces todos los NaNs de Lesividad faltantes por 'Se desconoce' y el código por 77.

In [ ]:
print(data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD'].unique())

In [ ]:
data_loader.data['datasets/accidentalidad']['LESIVIDAD'].where(data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD']==1)


In [ ]:
data_loader.data['datasets/accidentalidad']['LESIVIDAD'].where(data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD']==2)

In [ ]:
data_loader.data['datasets/accidentalidad']['LESIVIDAD'].where(data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD']==3)

In [ ]:
data_loader.data['datasets/accidentalidad']['LESIVIDAD'].where(data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD']==4)

In [ ]:
data_loader.data['datasets/accidentalidad']['LESIVIDAD'].where(data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD']==5)

In [ ]:
data_loader.data['datasets/accidentalidad']['LESIVIDAD'].where(data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD']==6)

In [ ]:
data_loader.data['datasets/accidentalidad']['LESIVIDAD'].where(data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD']==7)

In [ ]:
data_loader.data['datasets/accidentalidad']['LESIVIDAD'].where(data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD']==14)

In [ ]:
data_loader.data['datasets/accidentalidad']['LESIVIDAD'].where(data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD']==77)

In [ ]:
# Mapping dictionary
mapping_dict = {
    1: 'Atención en urgencias sin posterior ingreso',
    2: 'Ingreso inferior o igual a 24 horas',
    3: 'Ingreso superior a 24 horas',
    4: 'Fallecido 24 horas',
    5: 'Asistencia sanitaria ambulatoria con posterioridad',
    6: 'Asistencia sanitaria inmediata en centro de salud o mutua',
    7: 'Asistencia sanitaria sólo en el lugar del accidente',
    14: 'Sin asistencia sanitaria',
    77: 'Se desconoce',
}

# Fill missing values using the mapping dictionary
data_loader.data['datasets/accidentalidad']['LESIVIDAD'] = data_loader.data['datasets/accidentalidad']['LESIVIDAD'].fillna(data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD'].map(mapping_dict))
data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD'] = data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD'].fillna(data_loader.data['datasets/accidentalidad']['LESIVIDAD'].map({v: k for k, v in mapping_dict.items()}))

data_loader.data['datasets/accidentalidad']['LESIVIDAD'].fillna('Se desconoce', inplace=True)
data_loader.data['datasets/accidentalidad']['COD_LESIVIDAD'].fillna(77, inplace=True)


Por otro lado, positivo en droga tiene valor sólo si daba positivo, por lo que rellenar los valores faltantes con 0 es lo más lógico (siendo 0 negativo en droga). 

In [ ]:
data_loader.data['datasets/accidentalidad']['POSITIVA_DROGA'] = data_loader.data['datasets/accidentalidad']['POSITIVA_DROGA'].fillna(0)

Número, código de distrito, tipo de accidente, coordenadas (x e y), son atributos a los que sólo les falta un dato cada uno, por lo que no es representativo esta falta de datos y podemos rellenarlos con el valor más habitual. 

In [ ]:
numero = data_loader.data['datasets/accidentalidad']['COD_DISTRITO'].mode().iat[0]
codDistrito = data_loader.data['datasets/accidentalidad']['COD_DISTRITO'].mode().iat[0]
distrito = data_loader.data['datasets/accidentalidad']['COD_DISTRITO'].mode().iat[0]
accidente = data_loader.data['datasets/accidentalidad']['COD_DISTRITO'].mode().iat[0]
coorX = data_loader.data['datasets/accidentalidad']['COD_DISTRITO'].mode().iat[0]
coorY = data_loader.data['datasets/accidentalidad']['COD_DISTRITO'].mode().iat[0]


data_loader.data['datasets/accidentalidad']['NUMERO'] = data_loader.data['datasets/accidentalidad']['NUMERO'].fillna(numero)
data_loader.data['datasets/accidentalidad']['COD_DISTRITO'] = data_loader.data['datasets/accidentalidad']['COD_DISTRITO'].fillna(codDistrito)
data_loader.data['datasets/accidentalidad']['DISTRITO'] = data_loader.data['datasets/accidentalidad']['DISTRITO'].fillna(distrito)
data_loader.data['datasets/accidentalidad']['TIPO_ACCIDENTE'] = data_loader.data['datasets/accidentalidad']['TIPO_ACCIDENTE'].fillna(accidente)
data_loader.data['datasets/accidentalidad']['COORDENADA_X_UTM'] = data_loader.data['datasets/accidentalidad']['COORDENADA_X_UTM'].fillna(coorX)
data_loader.data['datasets/accidentalidad']['COORDENADA_Y_UTM'] = data_loader.data['datasets/accidentalidad']['COORDENADA_Y_UTM'].fillna(coorY)


Por último, donde queda dilema es en positivo alcohol, tipo de vehículo y estado meteorológico. En esta situación, lo más apropiado es ver si, relacionando estos atributos con algún otro, es más probable que los atributos valgan uno u otro valor.

- Estado meteorológico. Hay 7 valores posibles: despejado, lluvia débil, lluvia intensa, granizando, nevando, nublado, se desconoce. Aquí, por lo tanto, hay 3 vías de actuación:
    - Rellenar con "se desconoce", i.e.: ser fieles a lo que se sabe, reducir la proporción de datos artificiales (hay un 11% de datos faltantes), solución sencilla.
    - Rellenar con el valor más frecuente: despejado (representa el 75% de los datos), i.e.: solución con datos artificiales más sencilla.
    - Rellenar con valores aleatorios según la proporción en la que aparecen los datos, i.e.: el 75% de los datos faltantes se rellenan arbitrariamente con "Despejado".

    Lo que mejor preserva los datos es, rellenar con "se desconoce", pues la variable existe previamente.

In [ ]:
data_loader.data['datasets/accidentalidad']['ESTADO_METEOROLÓGICO'] = data_loader.data['datasets/accidentalidad']['ESTADO_METEOROLÓGICO'].fillna(pd.Series(np.random.choice(['Despejado', 'Lluvia débil', 'Lluvia moderada', 'Lluvia fuerte', 'Granizo', 'Nieve', 'Niebla', 'Se desconoce'], p=[0.7, 0.1, 0.05, 0.05, 0.025, 0.025, 0.025, 0.025], size=len(data_loader.data['datasets/accidentalidad']))))
data_loader.data['datasets/accidentalidad']['ESTADO_METEOROLÓGICO']

In [ ]:
data_loader.data['datasets/accidentalidad']['ESTADO_METEOROLÓGICO'] = data_loader.data['datasets/accidentalidad']['ESTADO_METEOROLÓGICO'].fillna('Se desconoce')

- Tipo de vehículo. Hay 34 valores posibles: Ambulancia SAMUR, autobús EMT, autobús, autobús articulado, autobús articulado EMT, autocaravana, bicicleta, bicicleta EPAC (pedaleo asistido), camión de bomberos, camión rígido, ciclo, ciclomotor, ciclomotor de dos ruedas L1e-B, cuadriciclo ligero, cuadriciclo no ligero, furgoneta, maquinaria de obras, microbús <= 17 plazas, moto de tres ruedas > 125cc, moto de tres ruedas hasta 125cc, motocicleta > 125cc, motocicleta hasta 125cc, otros vehículos con motor, otros vehículos sin motor, patinete no eléctrico, remolque, semirremolque, sin especificar, todo terreno, tractocamión, tren/metro, turismo (68%), VMU eléctrico, vehículo articulado. En esta variable hay 0.6% de valores faltantes, lo cual no es significativo, i.e.: la sustitución que elijamos tendrá menos repercusión en el estudio final. Aquí, por lo tanto, hay 2 vías de actuación:
    - Rellenar con "sin especificar", i.e.: solución sencilla y descriptiva pero que puede dar lugar a interpretaciones erróneas, pues puede haber sido otro tipo de vehículo que no se había registrado.
    - Rellenar con el valor más probable según otro atributo (por ejemplo, código de lesividad).
    
    La solución más apropiada es rellenar con el valor más probable según código de lesividad, por lo que vamos a ver primero cómo se relacionan ambos atributos y después rellenaremos los valores faltantes con el valor más probable.

In [ ]:
data_loader.data['datasets/accidentalidad']['TIPO_VEHICULO'] = data_loader.data['datasets/accidentalidad']['TIPO_VEHICULO'].fillna(data_loader.data['datasets/accidentalidad'].groupby('COD_LESIVIDAD')['TIPO_VEHICULO'].transform(lambda x:x.mode().iat[0]))

Para rellenar los datos de positivo en alcohol, usaremos lo más habitual según grupo de edad, sexo y lesividad de la persona

In [ ]:
data_loader.data['datasets/accidentalidad']["POSITIVA_ALCOHOL"] = data_loader.data['datasets/accidentalidad'].groupby(['RANGO_EDAD','SEXO','LESIVIDAD'])['POSITIVA_ALCOHOL'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty                                                                                                        else "Empty"))

Para determinar el tipo de persona, usaremos de criterio el rango de edad y la lesividad -si bien sólo faltan 3 datos, en esta situación podemos realizar un ajuste más específico-

In [ ]:
data_loader.data['datasets/accidentalidad']["TIPO_PERSONA"] = data_loader.data['datasets/accidentalidad'].groupby(['RANGO_EDAD','LESIVIDAD'])['TIPO_PERSONA'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty                                                                                                        else "Empty"))

In [ ]:
data_loader.data['datasets/accidentalidad'] = data_loader.data['datasets/accidentalidad'][data_loader.data['datasets/accidentalidad'].DISTRITO != '13.0']

Hacer una agrupación de los distritos de la zona norte y de la zona sur, calcular la desviación típica de las dos zonas y contar un poco lo que sale

In [ ]:
accidentes = data_loader.data['datasets/accidentalidad']

In [ ]:
accidentes['DISTRITO'] = accidentes['DISTRITO'].replace(['MORATALAZ', 'PUENTE-DE-VALLECAS', 'RETIRO', 'USERA', 'VILLA DE VALLECAS', 'VILLAVERDE', 'CARABANCHEL', 'VICÁLVARO', 'ARGANZUELA', 'CENTRO', 'LATINA', 'RETIRO'], 'SUR')
accidentes['DISTRITO'] = accidentes['DISTRITO'].replace(['CIUDAD LINEAL', 'HORTALEZA', 'BARAJAS', 'SALAMANCA', 'SAN BLAS-CANILLEJAS', 'FUENCARRAL-EL-PARDO', 'MONCLOA-ARAVACA', 'CHAMARTíN', 'CHAMBERí', 'TETUáN'], 'NORTE')

In [ ]:
mean = accidentes.mean()

In [ ]:
numAccidentes = accidentes.groupby('DISTRITO')

In [ ]:
std = numAccidentes.std()
accidentes['DISTRITO']['NORTE'].value_counts().std()
accidentes['DISTRITO']['SUR'].value_counts().std()

La desviación típica por distrito es 59640.214352398165, indica que hay una variabilidad alta en el número de accidentes entre cada distrito. Id est, el número de accidentes sube o baja 59640 accidentes respecto a la media (82838). Si separamos en zona norte y sur, vemos que hay una mayor dispersión de los datos en la zona sur que la zona norte (la desviación típica es mayor). Esto puede llevar a la conclusión de que es más fácil predecir la cantidad de accidentes que van a ocurrir en la zona norte que en la zona sur, aunque esto es lógico si también se tiene en cuenta que el número de accidentes en la zona sur es 3 veces la de la zona norte.